In [ ]:
import pathlib as pl
import matplotlib.pyplot as plt
import numpy as np
import flopy

In [ ]:
ws = pl.Path('./mf6_radstruct')
name = 'radstruct'
tempdir = pl.Path(ws, "temp")
tempdir.mkdir(parents=True, exist_ok=True)

In [ ]:
def h_analytical(r, r0, r1, h0, h1):
    rho = (1/r0 - 1/r) / (1/r0 - 1/r1)
    h = ((1 - rho) * h0 ** (13./3.) + rho * h1 ** (13./3.)) ** (3./13.)
    return h


In [ ]:
# make modflow model
exe_name = "/Users/langevin/langevin/dev/modflow6-fork.git/bin/mf6"
sim_ws = ws

Lx = 162000
Ly = 162000
nrow = 151
ncol = 151
dx = Lx / nrow
imid = int((nrow - 1) / 2)

x = np.linspace(0 + dx / 2., Lx - dx / 2., ncol)
y = np.linspace(Ly - dx / 2, dx / 2, nrow)
X, Y = np.meshgrid(x, y)

land_surface = 0.
roughness = 1.0

h1 = 1.0
h0 = h1 * 10
r0 = 10.
# r0 = 5000.
r1 = np.sqrt(2 * Lx ** 2)
r = np.sqrt((X - X[imid, imid]) ** 2 + (Y - Y[imid, imid]) ** 2)
h_analytical_solution = h_analytical(r, r0, r1, h0, h1)
h_analytical_solution[imid, imid] = h0

# Assign constant heads around perimeter of model
# and in a 9-cell block in the middle of the grid.
# Assign heads based on the analytical solution.
ibd = np.ones((nrow, ncol), dtype=int)
ibd[1:-1, 1:-1] = 0
ibd[imid-1:imid + 2, imid-1:imid + 2] = 1
rows, cols = np.where(ibd == 1)
spd_chd = [(i, j, h_analytical_solution[i, j]) for i, j in zip(rows, cols)]

nper = 1
perlen = nper * [1036800.0]
nstp = nper * [1]
tsmult = nper * [1]

tdis_rc = []
for i in range(nper):
    tdis_rc.append((perlen[i], nstp[i], tsmult[i]))

name = "swf"

# build MODFLOW 6 files
sim = flopy.mf6.MFSimulation(
    sim_name=f"{name}_sim",
    version="mf6",
    exe_name=exe_name,
    sim_ws=sim_ws,
)

# create tdis package
ats_filerecord = None
tdis = flopy.mf6.ModflowTdis(
    sim,
    ats_filerecord=ats_filerecord,
    time_units="SECONDS",
    nper=nper,
    perioddata=tdis_rc,
)

def add_model(sim, modelname, dev_swr_conductance):

    # surface water model
    swf = flopy.mf6.ModflowSwf(sim, modelname=modelname, save_flows=True)

    nouter, ninner = 500, 100
    hclose, rclose, relax = 1e-8, 1e-8, 0.97
    imsswf = flopy.mf6.ModflowIms(
        sim,
        print_option="SUMMARY",
        outer_dvclose=hclose,
        outer_maximum=nouter,
        under_relaxation="DBD",
        under_relaxation_theta=0.9,
        under_relaxation_kappa=0.0001,
        under_relaxation_gamma=0.0,
        inner_maximum=ninner,
        inner_dvclose=hclose,
        linear_acceleration="BICGSTAB",
        scaling_method="NONE",
        reordering_method="NONE",
        relaxation_factor=relax,
        preconditioner_levels=7,
        # backtracking_number=5,
        # backtracking_tolerance=1.0,
        # backtracking_reduction_factor=0.3,
        # backtracking_residual_limit=100.0,
        filename=f"{modelname}.ims",
    )
    sim.register_ims_package(imsswf, [swf.name])

    dis = flopy.mf6.ModflowSwfdis2D(
        swf,
        nrow=nrow,
        ncol=ncol,
        delr=dx,
        delc=dx,
        botm=land_surface,
        xorigin=0.,
    )

    dfw = flopy.mf6.ModflowSwfdfw(
        swf,
        dev_swr_conductance=dev_swr_conductance,
        print_flows=False,
        save_flows=True,
        save_velocity=True,
        manningsn=roughness,
        idcxs=None,
    )

    ic = flopy.mf6.ModflowSwfic(
        swf,
        strt=h0,
    )

    # output control
    oc = flopy.mf6.ModflowSwfoc(
        swf,
        budget_filerecord=f"{modelname}.bud",
        stage_filerecord=f"{modelname}.stage",
        saverecord=[
            ("STAGE", "ALL"),
            ("BUDGET", "ALL"),
        ],
        printrecord=[
            ("BUDGET", "ALL"),
        ],
    )

    chd = flopy.mf6.ModflowSwfchd(
        swf,
        maxbound=len(spd_chd),
        print_input=True,
        print_flows=True,
        stress_period_data=spd_chd,
    )

swfname = "swf_model"
add_model(sim, swfname, dev_swr_conductance=False)

# write the model files
sim.write_simulation()

# run the model
sim.run_simulation()

In [ ]:
modelgrid = flopy.discretization.StructuredGrid(
        nrow=nrow, 
        ncol=ncol,
        delr=np.array(ncol*[dx]),
        delc=np.array(nrow*[dx]),
    )
fpth = sim_ws / f"swf_model.stage"
stage_obj = flopy.utils.HeadFile(fpth, text="STAGE")
stage = stage_obj.get_data().flatten()

fpth = ws / f"{swfname}.bud"
cbb = flopy.utils.CellBudgetFile(fpth, precision="double")
velocity = cbb.get_data(text="DATA-VCOMP")[0]

nodes = stage.shape[0]
vx = np.ones((nodes), dtype=float) * 1.0e30
vy = np.ones((nodes), dtype=float) * 1.0e30
vz = np.ones((nodes), dtype=float) * 1.0e30
n0 = velocity["node"] - 1
vx[n0] = velocity["vx"]
vy[n0] = velocity["vy"]
vz[n0] = velocity["vz"]
vx = vx.reshape(nodes)
vy = vy.reshape(nodes)
vz = vz.reshape(nodes)


In [ ]:
# make a color map of fixed colors
from matplotlib import colors
cmap = colors.ListedColormap(['none', 'blue'])
bounds=[0, 1]
norm = colors.BoundaryNorm(bounds, cmap.N)
chd_array = np.ma.masked_where((ibd == 0), ibd)

with flopy.plot.styles.USGSPlot():
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(1, 1, 1)
    ax.set_aspect(1.)
    swf = sim.swf[0]
    pmv = flopy.plot.PlotMapView(modelgrid=modelgrid)
    qm = pmv.plot_array(chd_array, cmap=cmap, norm=norm)
    pmv.plot_grid(linewidths=0.1)
    ax.set_xlabel("X location, in meters")
    ax.set_ylabel("Y location, in meters")
    fig.savefig("../../doc/figures/oned-structured-grid.pdf", dpi=300)

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(1, 1, 1)
swf = sim.swf[0]
pmv = flopy.plot.PlotMapView(modelgrid=modelgrid)
cb = pmv.plot_array(stage)
plt.colorbar(cb, shrink=0.5)
if nrow < 50:
    ax.quiver(modelgrid.xcellcenters, modelgrid.ycellcenters, 
            vx, vy, scale=.05)
ax.set_title("Model")


In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(1, 1, 1)
swf = sim.swf[0]
pmv = flopy.plot.PlotMapView(modelgrid=modelgrid)
cb = pmv.plot_array(h_analytical_solution)
plt.colorbar(cb, shrink=0.5)
ax.set_title("Analytical")


In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(1, 1, 1)
swf = sim.swf[0]
pmv = flopy.plot.PlotMapView(modelgrid=modelgrid)
residual = stage - h_analytical_solution.flatten()
cb = pmv.plot_array(residual)
plt.colorbar(cb, shrink=0.5)
ax.set_title("Residual")

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(1, 1, 1)
    ax.set_aspect(1.)
    swf = sim.swf[0]
    pmv = flopy.plot.PlotMapView(modelgrid=modelgrid)
    hmin = stage.min()
    hmax = stage.max()
    # levels = np.linspace(hmin, hmax, 5)
    levels = [hmin + inc for inc in [0.01, 0.1, 0.25, 0.5, 1.]]
    levels = [1.17, 1.20, 1.45, 1.67, 2.]
    levels = np.arange(1.2, 2.2, .2)
    # levels = np.linspace(1, 9, 9)
    cs1 = pmv.contour_array(h_analytical_solution, levels=levels, colors="red", linewidths=2)
    cs2 = pmv.contour_array(stage, levels=levels, colors="black", linestyles="dotted", linewidths=2)
    clabels = ax.clabel(cs1, cs1.levels, inline=True, fontsize=10, colors="black")
    [txt.set_bbox(dict(facecolor='white', edgecolor='none', pad=0)) for txt in clabels]
    ax.set_xlabel("X location, in meters")
    ax.set_ylabel("Y location, in meters")
    fig.savefig("../../doc/figures/oned-structured-results.pdf", dpi=300)

In [ ]:
h_analytical_solution.min(), h_analytical_solution.max(), stage.min(), stage.max()